## Importing Libraries

In [1]:
import os
import shutil
import glob
from pathlib import Path
from dotenv import load_dotenv


import torch
from roboflow import Roboflow

## Setting Roboflow API Key

In [2]:
roboflow_api_key = os.getenv("roboflow_api_key")
rf = Roboflow(api_key=roboflow_api_key)

In [3]:
price_dataset_path = "./datasets/pricetags"

In [4]:
price_project = rf.workspace("andophine").project("pricetag-ohmyf")
price_dataset = price_project.version(1).download("yolov8", location=price_dataset_path, overwrite=True)


loading Roboflow workspace...
loading Roboflow project...


IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


Extracting Dataset Version Zip to ./datasets/pricetags in yolov8:: 100%|██████████| 1697/1697 [00:03<00:00, 439.49it/s]


In [5]:
pricetag_project = rf.workspace("oscar6207-gmail-com").project("tag-gaknf")
pricetag_dataset = pricetag_project.version(1).download("yolov8", location=price_dataset_path+"2")

loading Roboflow workspace...
loading Roboflow project...


In [6]:
pricetag_project2 = rf.workspace("miraisouzou2").project("-2-rctlm")
pricetag_dataset2 = pricetag_project2.version(1).download("yolov8", location=price_dataset_path+"3")

loading Roboflow workspace...
loading Roboflow project...


In [7]:
pricetag_project3 = rf.workspace("juan-ignacio-villaln-luis").project("tags-vz8np")
pricetag_dataset3 = pricetag_project3.version(2).download("yolov8", location=price_dataset_path+"4")

loading Roboflow workspace...
loading Roboflow project...


## Merge Datasets

In [8]:
complete_dataset_path = "./datasets/pricetags_total"

In [10]:
for split in ["train", "test", "valid"]:
    os.makedirs(f"{complete_dataset_path}/{split}/images", exist_ok=True)
    os.makedirs(f"{complete_dataset_path}/{split}/labels", exist_ok=True)

In [ ]:
def merge_datasets(dataset_paths, complete_dataset_path):
    for dataset_path in dataset_paths:
        for split in ["train", "test", "valid"]:
            for file_path in glob.glob(dataset_path + f"/{split}/images/*.jpg"):
                shutil.copy(file_path, f"{complete_dataset_path}/{split}/images")
            for file_path in glob.glob(dataset_path + f"/{split}/labels/*.txt"):
                shutil.copy(file_path, f"{complete_dataset_path}/{split}/labels")

In [ ]:
merge_datasets([price_dataset_path, price_dataset_path + "2", price_dataset_path + "3", price_dataset_path + "4"], complete_dataset_path)

In [ ]:
with open(complete_dataset_path + "/data.yaml", "w") as fh:
    data = """
names:
- pricetag
nc: 1
path: /Users/priyanshutuli/Desktop/Planogram_Optimization/datasets/pricetags_total
test: ./test/images
train: ./train/images
val: ./valid/images
\n
    """
    fh.write(data)

## Yolo V8 Object Detection

In [11]:
from ultralytics import YOLO
from ultralytics.engine import trainer

In [12]:
model_medium = YOLO('yolov8m.pt', task="detect")

In [13]:
seed = 0
device = "cpu"
image_size = 640
batch_size = 32

In [14]:
data_yaml_path = "/Users/priyanshutuli/Desktop/Planogram_Optimization/datasets/pricetags_total/data.yaml"

In [15]:
checkpoint_path = "./runs/detect"
name="finetune_pricetag_model"
os.makedirs(checkpoint_path, exist_ok=True)

In [58]:
model_medium.train(data=data_yaml_path, epochs=50, imgsz=image_size, seed=seed, device=device, save=True, patience=10, batch=batch_size, verbose=True,
                      plots=True, cos_lr=True, lr0=1e-4, lrf=1e-6, optimizer='AdamW', deterministic=True, save_period=1, amp=True, val=True, augment=True, dropout=0.3, freeze=10, project=checkpoint_path, name=name)


New https://pypi.org/project/ultralytics/8.1.45 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.11 torch-2.2.1 CPU (Apple M1)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/Users/priyanshutuli/Desktop/Planogram_Optimization/datasets/pricetags_total/data.yaml, epochs=50, patience=10, batch=32, imgsz=640, save=True, save_period=1, cache=False, device=cpu, workers=8, project=./runs/detect, name=finetune_pricetag_model, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, overlap_mask=True, mask_ratio=4, dropout=0.3, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_b

In [22]:
last_checkpoint_path = "/Users/priyanshutuli/Desktop/Planogram_Optimization/runs/detect/finetune_pricetag_model5/weights/last.pt"
model_resume_training_froozen_layers = YOLO(last_checkpoint_path, task="detect")

In [23]:
model_resume_training_froozen_layers.train(resume=True, epochs=25)

New https://pypi.org/project/ultralytics/8.1.45 available 😃 Update with 'pip install -U ultralytics'
Python(67910) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Ultralytics YOLOv8.0.196 🚀 Python-3.10.11 torch-2.2.1 CPU (Apple M1)
engine/trainer: task=detect, mode=train, model=/Users/priyanshutuli/Desktop/Planogram_Optimization/runs/detect/finetune_pricetag_model5/weights/last.pt, data=/Users/priyanshutuli/Desktop/Planogram_Optimization/datasets/pricetags_total/data.yaml, epochs=50, patience=10, batch=32, imgsz=640, save=True, save_period=1, cache=False, device=cpu, workers=0, project=./runs/detect, name=finetune_pricetag_model, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, overlap_mask=True, mask_ratio=4, dropout=0.3, val=True, split=val, save_json=False, save_hybrid=False, conf=No

KeyboardInterrupt: 

In [19]:
inference_images_path = "./inference_images"
save_inference_path = "./runs/detect/"
inference_name="finetune_pricetag_model_predictions"

In [24]:
results = model_resume_training_froozen_layers.predict(inference_images_path, conf=0.5, imgsz=image_size, iou=0.7, save=True, device=device, seed=seed, project = save_inference_path, name=inference_name)

Python(74131) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.

image 1/998 /Users/priyanshutuli/Desktop/Planogram_Optimization/inference_images/0205BB3A-F171-4F92-A552-46B64F835D30_IMG_3467.jpeg: 640x480 (no detections), 309.1ms
image 2/998 /Users/priyanshutuli/Desktop/Planogram_Optimization/inference_images/0F488364-219C-4A21-94B2-0B6A884420DD_IMG_0041.jpeg: 640x480 2 pricetags, 232.2ms
image 3/998 /Users/priyanshutuli/Desktop/Planogram_Optimization/inference_images/12B22766-4862-470A-BC05-DAB41EFB537D_IMG_0986.jpeg: 640x480 (no detections), 316.5ms
image 4/998 /Users/priyanshutuli/Desktop/Planogram_Optimization/inference_images/177E5396-E9E2-47EC-B481-AA787E402CC7_IMG_1759.jpeg: 640x480 3 pricetags, 263.6ms
image 5/998 /Users/priyanshutuli/Desktop/Planogram_Optimization/inference_images/1AE868B4-02F6-4F6D-B09C-3B65042035D7_IMG_3525.jpeg: 640x480 3 pricetags, 264.6ms
image 6/998 /Users/priyanshutuli/Desktop/Planogram_Optimization/inference_images/20